In [ ]:
# Narayana Tangudu

import numpy as np  
import pandas as pd  

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv("../input/auto-insurance-claims-data/insurance_claims.csv")
df1.head()

In [ ]:
df1.drop('_c39', inplace=True, axis = 1)

In [ ]:
df1.describe()

In [ ]:
print ("Rows     : " ,df1.shape[0])
print ("Columns  : " ,df1.shape[1])
print ("\nFeatures : \n" ,df1.columns.tolist())
print ("\nMissing values :  ", df1.isnull().sum().values.sum())
print ("\nUnique values :  \n",df1.nunique())

In [ ]:
df1.duplicated()

In [ ]:
df1.dtypes

In [ ]:
for col in [ 'policy_state', 'policy_csl', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'incident_date',
            'incident_type', 'collision_type', 'incident_severity', 'authorities_contacted', 'incident_state', 'incident_city', 'incident_location',
             'property_damage',  'police_report_available','vehicle_claim', 'auto_make', 'auto_model', 'fraud_reported']:
    df1[col] = df1[col].astype('category')

In [ ]:
df1.dtypes

In [ ]:
df1['policy_bind_date'] = pd.to_datetime(df1['policy_bind_date'], format='%Y-%m-%d')

In [ ]:
df1.dtypes

In [ ]:
df1.head(2)

In [ ]:
df1['policy_number'].value_counts().sort_values()
#policy numbers seems unique and hence dropping this feature

In [ ]:
df1['fraud_reported'].value_counts().sort_values()

In [ ]:
df1.drop('policy_number', axis = 1, inplace =True)

In [ ]:
df1.replace('?','No info')

In [ ]:
df1.groupby(['insured_sex'])['fraud_reported'].count().sort_values().plot(kind='bar')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df1.total_claim_amount)
plt.title("Histogram of total amount claimed by the customers")
plt.xlabel("Amount claimed")
plt.ylabel("Number of customers")

highest claimed amount is 60k-70k

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df1.age)
plt.title("Histogram of age of the customers")
plt.xlabel("Age of the customers")
plt.ylabel("Number of customers")

In [ ]:
df1.groupby(['insured_sex','age']).fraud_reported.count().plot(kind='bar',figsize=(30,5))

Middle aged females reporting more fraud cases

In [ ]:
import seaborn as sns
sns.countplot(df1['fraud_reported']);

In [ ]:
df1.groupby(['fraud_reported','age']).age.count()

In [ ]:
plt.bar(df1['age'], df1['fraud_reported']== 'Y')
plt.title("Age of the customers")
plt.xlabel("Age of the customers")
plt.ylabel("Fraud_reported")

In [ ]:
plt.bar(df1['age'], df1['fraud_reported']== 'N')
plt.title("Age of the customers")
plt.xlabel("Age of the customers")
plt.ylabel("Fraud_reported")

In [ ]:
abc = df1.groupby([ 'auto_model'])['fraud_reported'].count().plot(kind='bar',figsize=(30, 5), rot=90)

In [ ]:
import seaborn as sns
sns.set(style="darkgrid")

ax = sns.countplot(x="age", data=df1)

In [ ]:
df1.groupby(['policy_state']).fraud_reported.value_counts().plot(kind='bar',figsize=(20, 5), rot=90)
plt.xlabel("State")
plt.ylabel("Fraud Reported")
plt.title("Distribution of fraud cases w.r.t states")
plt.show()

In [ ]:
#which areas suspecting what type of outages
pd.crosstab(df1['auto_model'],df1['vehicle_claim'])

In [ ]:
#count of frauds reported w.r.t auto model
pd.crosstab(df1['auto_model'],df1['fraud_reported'])

In [ ]:
df1.shape

In [ ]:
X = df1.iloc[:,:-1]
Y = df1.iloc[:, -1]
# here Y has column as 'fraud_reported' which is the last column in the data frame df1

categorical_data = X.select_dtypes(exclude="number")
categorical_cols = categorical_data.columns

X = X.drop(categorical_cols, axis=1)
print(X.shape)
print(Y.shape)


In [ ]:
X.dtypes

In [ ]:
Y.dtypes

In [ ]:
X.info

In [ ]:
from sklearn.model_selection import train_test_split
one_hot_data = pd.get_dummies(categorical_data)
X = X.join(one_hot_data)

Xcols = X.columns
ycols = Y

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, stratify=Y)

In [ ]:
Xcols

In [ ]:
ycols

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape,y_test.shape

In [ ]:
x_train.columns

In [ ]:
categorical_cols

In [ ]:
x_train = x_train.drop(['policy_bind_date','auto_year'], axis = 1)
x_test = x_test.drop(['policy_bind_date','auto_year'], axis = 1)

In [ ]:
num_cols = ['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'insured_zip',  'incident_hour_of_the_day','capital-loss','capital-gains'
       'number_of_vehicles_involved',  'bodily_injuries',
       'witnesses',  'total_claim_amount','injury_claim', 'property_claim']

In [ ]:
from sklearn.preprocessing import StandardScaler
 
print('scaling numerical columns')
scaler = StandardScaler(num_cols)
x_train= scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
print("Done with standard scaling, Proceed.....All the best :)")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  cross_val_score 
Logistic_model = LogisticRegression(random_state=200)
Logistic_model.fit( x_train,y_train)

y_pred1_train = Logistic_model.predict(x_train)
y_pred1_test = Logistic_model.predict(x_test)

from sklearn.metrics import accuracy_score
print("Accuracy on Train using Logistic Model is:", accuracy_score(y_train, y_pred1_train))
print("Accuracy on Test using Logistic Model is:", accuracy_score(y_test, y_pred1_test))

scores = cross_val_score(Logistic_model, x_train,y_train, cv=5, scoring='accuracy')
print("Max cross validation score on logistic model",max(scores))
print("Mean cross validation score on logistic model",np.mean(scores))
print("Min cross validation score on logistic model",min(scores))

# Decision trees with cross validation


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

DT_clf = DecisionTreeClassifier(criterion='entropy', max_depth=800,max_features=15,min_samples_split=2,min_samples_leaf=1,class_weight='balanced')

# Train Decision Tree Classifer
DT_clf = DT_clf.fit(x_train, y_train)

#Predict the response for train and validation dataset
y_pred2_train = DT_clf .predict(x_train)
y_pred2_test = DT_clf.predict(x_test)


conf_matrix = confusion_matrix(y_pred2_test, y_test)
plot_confusion_matrix(conf_matrix)

precision = precision_score(y_pred2_test, y_test, pos_label='Y')
recall = recall_score(y_pred2_test, y_test, pos_label='Y')


print("on complete data using DT:\n")
print("Accuracy on Train using Decision Tree Model is:", accuracy_score(y_train, y_pred2_train))
print("Accuracy on Test using Decision Tree Model is:", accuracy_score(y_test, y_pred2_test))
scores = cross_val_score(DT_clf,x_train, y_train, cv=5, scoring='accuracy')
print("Max cross validation score on DT model",max(scores))
print("Mean cross validation score on DTmodel",np.mean(scores))
print("Min cross validation score on DT model",min(scores))

# Using Smote on train data**

In [ ]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote=SMOTE("minority")
x_smote,y_smote = smote.fit_sample(x_train, y_train)

# Decision tree model on smote data

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

DT_clf_smote = DecisionTreeClassifier(criterion='entropy', max_depth=800,max_features=15,min_samples_split=2,min_samples_leaf=1,class_weight='balanced')

# Train Decision Tree Classifer
DT_clf_smote = DT_clf_smote.fit(x_smote, y_smote)

#Predict the response for train and validation dataset
y_pred2_train_smote = DT_clf_smote.predict(x_smote)
y_pred2_test_smote = DT_clf_smote.predict(x_test)


conf_matrix_smote = confusion_matrix(y_pred2_test_smote, y_test)
plot_confusion_matrix(conf_matrix_smote)

precision = precision_score(y_pred2_test_smote, y_test, pos_label='Y')
recall = recall_score(y_pred2_test_smote, y_test, pos_label='Y')


print("on complete data using DT:\n")
print("Accuracy on Train using Decision Tree Model is:", accuracy_score(y_smote, y_pred2_train_smote))
print("Accuracy on Test using Decision Tree Model is:", accuracy_score(y_test, y_pred2_test_smote))
scores = cross_val_score(DT_clf,x_smote, y_smote, cv=5, scoring='accuracy')
print("Max cross validation score on DT model",max(scores))
print("Mean cross validation score on DTmodel",np.mean(scores))
print("Min cross validation score on DT model",min(scores))

# Random Forest on smote data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC_model = RandomForestClassifier(random_state = 1, max_features='sqrt',oob_score=True, n_estimators = 1200)
RFC_model.fit(x_smote, y_smote)
y_pred3_trainSMOTE = RFC_model.predict(x_smote)

y_pred3_testSMOTE = RFC_model.predict(x_test)
print("ON SMOTE dataset:\n")

conf_matrix_smote = confusion_matrix(y_pred3_testSMOTE, y_test)
plot_confusion_matrix(conf_matrix_smote)

precision = precision_score(y_pred3_testSMOTE, y_test, pos_label='Y')
recall = recall_score(y_pred3_testSMOTE, y_test, pos_label='Y')


print("on complete data using random Forest:\n")
print("Accuracy on Train using random Forest Model is:", accuracy_score(y_smote, y_pred3_trainSMOTE))
print("Accuracy on Test using random Forest Model is:", accuracy_score(y_test, y_pred3_testSMOTE))
scores = cross_val_score(DT_clf,x_smote, y_smote, cv=5, scoring='accuracy')
print("Max cross validation score on random Forest model",max(scores))
print("Mean cross validation score on random Forestmodel",np.mean(scores))
print("Min cross validation score on random Forest model",min(scores))
print("oob score", RFC_model.oob_score_)

Please Note: Though the cross validation scores showing model is stable but accuracy on test is very low, 
    the size of the given data must be the reason for such performance.

model DT giving best accuracy scores .